## Solving Poisson on the Tesserakt Finite Element Space 
First we construct a fully structured 4 dimensional mesh. This is obviously a very specific group of $\Omega$ domains we can approximate in this way but it is still a useful construction.

In [ ]:
from import_hack import *
from methodsnm.mesh_4d import *
from methodsnm.visualize import *
import math
import numpy as np
np.math = math
m=4
mesh = StructuredTesseraktMesh(m,m,m,m)
for elnr, verts in enumerate(mesh.elements()):
    print(elnr,verts)
    trafo = mesh.trafo(elnr)
    print(trafo.map(np.array([0.5,0.5,0.5,0.5])))
    if elnr == 3:
        break
print("Number of Vertices",len(mesh.vertices))
print("Number of edges ",len(mesh.edges))
print("Number of boundary edges" ,len(mesh.bndry_edges))
print("Number of faces",len(mesh.faces))
print("Number of boundary faces",len(mesh.bndry_faces))
print("Number of volumes",len(mesh.volumes))
print("Number of boundary volumes",len(mesh.bndry_volumes))

source module for methodsNM imported.
0 [  0   1   5   6  25  26  30  31 125 126 130 131 150 151 155 156]
[0.125 0.125 0.125 0.125]
1 [125 126 130 131 150 151 155 156 250 251 255 256 275 276 280 281]
[0.125 0.125 0.125 0.375]
2 [250 251 255 256 275 276 280 281 375 376 380 381 400 401 405 406]
[0.125 0.125 0.125 0.625]
3 [375 376 380 381 400 401 405 406 500 501 505 506 525 526 530 531]
[0.125 0.125 0.125 0.875]
4 [ 25  26  30  31  50  51  55  56 150 151 155 156 175 176 180 181]
[0.125 0.125 0.375 0.125]
5 [150 151 155 156 175 176 180 181 275 276 280 281 300 301 305 306]
[0.125 0.125 0.375 0.375]
6 [275 276 280 281 300 301 305 306 400 401 405 406 425 426 430 431]
[0.125 0.125 0.375 0.625]
Number of Vertices 625
Number of edges  2000
Number of boundary edges 1568
Number of faces 2400
Number of boundary faces 1498
Number of volumes 1280
Number of boundary volumes 512


Most of the numbers above are eazily checkable

In [39]:
from methodsnm.fes import *
fes = P1_Tesserakt_Space(mesh)
print(fes.ndof)

625


As always we have a guidance example for now we will solve the purely spacial $ -\Delta u_x =f $ problem on $\Omega = [0,1]^4$. This is a preparation for the con-diffusion problem we will solve [here](../convection-diffusion/space_time_4D.ipynb)

In [40]:
from numpy import sin, pi, exp
def list_diff(a, b):
    """Entfernt alle Elemente aus Liste a, die in Liste b enthalten sind."""
    return [x for x in a if x not in b]
top= mesh.filter_bndry_points("max",3)
bottom= mesh.filter_bndry_points("min",3)
bndry = list_diff(mesh.bndry_vertices,top)
freedofs = list_diff(mesh.vertices,bndry)

ud = lambda x: sin(pi*x[0])*sin(pi*x[1])*sin(pi*x[2])
uh = FEFunction(fes)
uh._set(ud,bndry=bndry)

In [41]:
from methodsnm.forms import *
from methodsnm.formint import *
BF = BilinearForm(fes)
c = GlobalFunction(lambda x: 1, mesh = mesh)
BF += LaplaceIntegral_without_time(c)
BF += TimeIntegral(c)
BF.assemble()

LF = LinearForm(fes)
LF.assemble()


In [42]:
from methodsnm.solver import *
res = LF.vector - BF.matrix @ uh.vector
uh.vector += solve_on_freedofs(BF.matrix, res, freedofs)

In [43]:
uex = lambda x: exp(-3*pi**2*x[3])*sin(pi*x[0])*sin(pi*x[1])*sin(pi*x[2])
uexact = FEFunction(fes)
uexact._set(uex)

In [44]:
from methodsnm.forms import compute_difference_L2
l2diff = compute_difference_L2(uh, uexact, mesh, intorder = 3)
print("l2diff =", l2diff)

l2diff = 0.042011221850749514
